## Session 4 : Data Manipulation

Exercise 1- Using cpi.csv data, perform the below tasks

Data is coming from the FRED Website : https://fred.stlouisfed.org/series/CORESTICKM159SFRBATL
- Display first 7 rows
- Display bottom 10 rows
- Check the type of CPI column, and type of data inside the column
- Compute below statistics
    - Mean cpi
    - Min/Max cpi
    - Standard Deviation
    - Print the summary of describe data

In [ ]:
import pandas as pd
df_cpi = pd.read_csv('cpi.csv')

# Display first 7 rows
df_cpi.head(7)

# Display bottom 10 rows
df_cpi.tail(10)

# Check type of CPI column
type(df_cpi.CPI)
type(df_cpi.CPI.iloc[0])
print(df_cpi.dtypes)

# Compute mean, min, max, std
mean_cpi = df_cpi['CPI'].mean()
min_cpi = df_cpi['CPI'].min()
max_cpi = df_cpi['CPI'].max()
std_cpi = df_cpi['CPI'].std()

# Print Summary describe data
df_cpi.describe()

,CPI
count,696.000000
mean,4.309990
std,2.656953
min,0.663868
25%,2.472013
50%,3.370944
75%,5.142048
max,15.774167


Exercise 2 - Using the unemployment_rate.csv data, (https://fred.stlouisfed.org/series/UNRATE)
1. Check the type of the index of unrate
2. Select unemployment data between 2015 and 2020
3. Find the maximum unemployment rate during that period
4. Print the min and max value on the period with the corresponding date in ISO format (YYYY-MM-DD)

In [11]:
from datetime import datetime as dt
unrate_df = pd.read_csv("unemployment_rate.csv")

# Check the type of 
print(type(unrate_df.index[0]))
print(type(unrate_df['date'].iloc[0]))

# Convert date to datetime
unrate_df['date'] = pd.to_datetime(unrate_df['date'])

# Set date as index
unrate_df = unrate_df.set_index('date')
# unrate_df.set_index('date', inplace=True)

# Select unemployment data from 2015 to 2020
unrate_df = unrate_df[unrate_df['UNRATE'] != '.']
unrate_df['UNRATE'] = unrate_df['UNRATE'].astype(float)
unrate_2015_2020_df = unrate_df['2015':'2020']

# Find min/max
# Minimum value and its index

min_value = unrate_2015_2020_df['UNRATE'].min()
min_index = unrate_2015_2020_df['UNRATE'].idxmin()

# Maximum value and its index
max_value = unrate_2015_2020_df['UNRATE'].max()
max_index = unrate_2015_2020_df['UNRATE'].idxmax()

print("Min unrate was ", min_value, " on ", dt.strftime(min_index, "%Y-%m-%d"))
print("Max unrate was ", max_value, " on ", max_index)

<class 'int'>
<class 'str'>
Min unrate was  3.5  on  2019-09-01
Max unrate was  14.8  on  2020-04-01 00:00:00


Exercise 3 - Data Cleaning and Frequency
1. Check the frequency of:
    - GDP (https://fred.stlouisfed.org/series/GDP)
    - Unemployment
    - CPI
2. Create a function to set date as pd.datetime, set date as index, clean up the main values. Function should have 3 inputs : dataFrame to clean up, the column names of the values to clean, the column name of the date column
3. Convert unemployment and CPI to quarterly averages using pandas functionnality under a function

In [ ]:
from typing import Any, Union, Iterable
def dataFrame_clean_up(df:pd.DataFrame, values_col:str, date_col:str='date') -> pd.DataFrame:
    # 1. Turn date into dateTime
    df[date_col] = pd.to_datetime(df[date_col])

    # 2. Set date col as index
    df.set_index(date_col, inplace=True)

    # 3. Clean up data and turn value to float
    cols = [values_col]
    for col in cols:
        df = df[df[col]!= "."]
        df[col] = df[col].astype(float)
    
    return df

def quarterly_resampling(df:pd.DataFrame) -> pd.DataFrame:
    df = df.resample('QE').mean()
    df.index = df.index + pd.offsets.MonthBegin(1)
    return df

In [ ]:
gdp_df = pd.read_csv("gdp.csv")
unrate_df = pd.read_csv("unemployment_rate.csv")
cpi_df = pd.read_csv("cpi.csv")

gdp_df = dataFrame_clean_up(gdp_df, 'GDP')
unrate_df = dataFrame_clean_up(unrate_df, 'UNRATE')
cpi_df = dataFrame_clean_up(cpi_df, 'CPI')

print('gdp_df: ', gdp_df.index) # Quarterly
print('unrate_df: ', unrate_df.index) # Montly
print('cpi_df: ', cpi_df.index) # Montly

# Resample data
unrate_df = quarterly_resampling(unrate_df)
cpi_df = quarterly_resampling(cpi_df)

Exercise 4 - Creating Economic Indicators
1. Create GDP growth rate (%)
2. Compute unemployment rate (%) from unemployment level (https://fred.stlouisfed.org/series/UNEMPLOY)

In [ ]:
gdp_df = pd.read_csv("gdp.csv")
unlevel_df = pd.read_csv("unemployment_level.csv")

# Clean data
gdp_df = dataFrame_clean_up(gdp_df, 'GDP')
unlevel_df = dataFrame_clean_up(unlevel_df, 'UNLEVEL')

gdp_df["GDP_growth"] = gdp_df["GDP"].pct_change() * 100
unlevel_df["unrate"] = unlevel_df["UNLEVEL"].pct_change() * 100


Exercice 5 - Concat, join manipulation
1. Merge cpi and unrate on their index
2. Find a pandas methods to fill forward NaN values with latest available value, and drop na
3. Resample raw cpi and unrate data to quartly data, try to concat it with GDP
4. Inner Join dataFrame from 2. with GDP Growth data


In [ ]:
gdp_df = pd.read_csv("gdp.csv")
unrate_df = pd.read_csv("unemployment_rate.csv")
cpi_df = pd.read_csv("cpi.csv")

# Clean data
gdp_df = dataFrame_clean_up(gdp_df, 'GDP')
unrate_df = dataFrame_clean_up(unrate_df, 'UNRATE')
cpi_df = dataFrame_clean_up(cpi_df, 'CPI')

# Merge df
merged_df = unrate_df.join(cpi_df, how='outer')

# Handle NaN
merged_df.ffill(inplace=True)
merged_df.dropna(inplace=True)

# Resample
unrate_df = quarterly_resampling(unrate_df)
cpi_df = quarterly_resampling(cpi_df)

gdp_df["GDP"] = gdp_df["GDP"].pct_change() * 100
concat_df = pd.concat([unrate_df, cpi_df, gdp_df])

merged_df = merged_df.join(gdp_df, how='inner')

Exercise 6 - Time Series Plot (Plotly)
Using the previous merged dataFrame and Plotly documentation (https://plotly.com/python/basic-charts/)
1. Plot GDP growth and inflration on the same graph
2. Add a title and labels
3. Identify major economic shocks visually

Exercise 7 — Economic Relationship (Scatter Plot)
1. Create a scatter plot:
    - x-axis: Unemployment rate
    - y-axis: GDP growth
2. Add a trend line
3. Interpret the result

Exercise 8 — Correlation Matrix
1. Compute the correlation matrix for:
    - GDP growth
    - Inflation
    - Unemployment
2. Plot it using a heatmap
3. interpret the strongest correlation